Semiconductor Thermal Analysis with GaN Losses
This notebook runs an IcePak thermal simulation using power loss data from a GaN transistor-based power converter.

In [ ]:
import os
import json
import csv
from ansys.aedt.core import Icepak
import matplotlib.pyplot as plt
import numpy as np


## User Configuration
# Modify these settings to match your project requirements



In [ ]:
# Set up your file paths
project_folder = "YOUR_PROJECT_FOLDER_PATH"  # Directory where the project will be saved
idf_file = "YOUR_IDF_FILE_PATH"              # Path to the IDF file
bdf_file = "YOUR_BDF_FILE_PATH"              # Path to the BDF file
edb_file = "YOUR_EDB_FILE_PATH"              # Path to the EDB file
gan_losses_json = "gan_losses.json"          # Path to the GaN losses JSON file
output_csv = "thermal_results.csv"           # Path to the output CSV file

# AEDT configuration
aedt_version = "2024.2"                      # AEDT version to use
ng_mode = False

In [ ]:
# Device designators for thermal simulation
# Map the devices in your design to the devices in the GaN losses file
device_mapping = {
    "Q1": "GaN_1",  # Device designator in IcePak : Name to use in results
    # Add more devices as needed
    # "Q2": "GaN_2",
}


In [ ]:
# Load the GaN losses data from JSON file
def load_gan_losses(json_file):
    try:
        with open(json_file, 'r') as f:
            gan_data = json.load(f)
        print("Successfully loaded GaN losses data:")
        print(f"- Operating conditions: Vin={gan_data['operating_conditions']['Vin']}V, "
              f"Vout={gan_data['operating_conditions']['Vout']}V, "
              f"Fsw={gan_data['operating_conditions']['Switching_frequency']/1000}kHz")
        print(f"- Total loss: {gan_data['losses']['total_loss_W']}W")
        print(f"- Efficiency: {gan_data['efficiency']['efficiency_percent']}%")
        return gan_data
    except Exception as e:
        print(f"Error loading GaN losses data: {e}")
        return None

# Load the GaN losses data
gan_data = load_gan_losses(gan_losses_json)

In [ ]:
# Create device losses dictionary for IcePak
device_losses = {}
if gan_data:
    # For this example, we'll use the total loss from the GaN data for all devices
    # You could also distribute different loss components to different devices
    total_loss = gan_data['losses']['total_loss_W']

    # Assign the same loss to all devices (modify as needed for your application)
    for designator in device_mapping:
        device_losses[designator] = f"{total_loss}W"

    print("\nDevice losses for thermal simulation:")
    for designator, loss in device_losses.items():
        print(f"- {designator}: {loss}")


Define Core Thermal Simulation Function

In [ ]:
def setup_solve_and_export_results(
    project_folder, idf_file, bdf_file, edb_file, device_losses, output_csv, aedt_version="2024.2", ng_mode=False
):
    """
    Sets up an IcePak simulation, assigns power losses to devices, solves the simulation, and exports results.

    Parameters:
    - project_folder (str): Directory where the project will be saved.
    - idf_file (str): Path to the IDF file.
    - bdf_file (str): Path to the BDF file.
    - edb_file (str): Path to the EDB file.
    - device_losses (dict): Dictionary of device designators and their power losses.
                            Example: {"Q1": "5W", "D1": "3W"}
    - output_csv (str): Path to the output CSV file.
    - aedt_version (str): AEDT version to use. Default is "2024.2".
    - ng_mode (bool): Non-graphical mode. Default is False.
    """
    # Ensure the project folder exists
    if not os.path.exists(project_folder):
        os.makedirs(project_folder)

    # Define the project path
    project_path = os.path.join(project_folder, "Icepak_Project.aedt")

    # Launch IcePak
    ipk = Icepak(
        project=project_path,
        version=aedt_version,
        new_desktop=True,
        non_graphical=ng_mode,
    )

    try:
        # Import IDF and BDF files
        print("Importing IDF and BDF files...")
        ipk.import_idf(board_path=bdf_file)

        # Save the project after IDF import
        ipk.save_project()

        # Import the EDB file using HFSS 3D Layout
        print("Importing EDB file...")
        hfss3d_lo = ipk.create_3dlayout_project(edb_path=edb_file)
        hfss3d_lo.save_project()

        # Link EDB to IcePak
        print("Linking EDB file to IcePak...")
        ipk.create_pcb_from_3dlayout(
            component_name="PCB_pyAEDT",
            project_name=hfss3d_lo.project_file,
            design_name=hfss3d_lo.design_name,
            extent_type="Polygon",
            outline_polygon="poly_0",
            power_in=0,
            close_linked_project_after_import=False,
        )

        # Assign power losses to specific devices and define monitor points
        print("Assigning power losses and defining monitors...")
        monitors = []
        for designator, power in device_losses.items():
            device = ipk.modeler[designator]
            if device:
                # Assign power loss
                ipk.create_source_block(object_name=designator, input_power=power)

                # Automatically identify the top and bottom faces
                faces = device.faces
                top_face = max(faces, key=lambda f: f.center[2])  # Highest Z coordinate
                bottom_face = min(faces, key=lambda f: f.center[2])  # Lowest Z coordinate

                # Add monitors to the top and bottom faces
                top_monitor = ipk.monitor.assign_face_monitor(
                    face_id=top_face.id,
                    monitor_quantity="Temperature",
                    monitor_name=f"{designator}_Top_Monitor",
                )
                bottom_monitor = ipk.monitor.assign_face_monitor(
                    face_id=bottom_face.id,
                    monitor_quantity="Temperature",
                    monitor_name=f"{designator}_Bottom_Monitor",
                )
                monitors.append({
                    "designator": designator,
                    "device_name": device_mapping.get(designator, designator),
                    "top": top_monitor,
                    "bottom": bottom_monitor
                })
            else:
                print(f"Warning: Device '{designator}' not found in the model.")

        # Setup the project for natural convection
        print("Setting up the simulation...")
        setup = ipk.create_setup()
        setup.props["Flow Regime"] = "Turbulent"
        setup.props["Ambient Temperature"] = "20cel"  # Ambient temperature
        setup.props["Convergence Criteria - Max Iterations"] = 100
        ipk.save_project()

        # Solve the project
        print("Solving the simulation...")
        ipk.analyze(setup=setup.name)

        # Extract monitor temperatures
        print("Extracting results...")
        results = []
        for monitor in monitors:
            top_temp = ipk.post.evaluate_monitor_quantity(monitor=monitor["top"], quantity="Temperature")["Mean"]
            bottom_temp = ipk.post.evaluate_monitor_quantity(monitor=monitor["bottom"], quantity="Temperature")["Mean"]
            power_loss = float(device_losses[monitor["designator"]][:-1])  # Remove "W" from the power loss
            temp_rise = top_temp - bottom_temp
            psi = temp_rise / power_loss

            results.append({
                "Device": monitor["device_name"],
                "Designator": monitor["designator"],
                "Power Loss (W)": power_loss,
                "Top Temp (C)": top_temp,
                "Bottom Temp (C)": bottom_temp,
                "Temp Rise (C)": temp_rise,
                "Psi (K/W)": psi
            })

        # Save results to CSV
        print(f"Saving results to {output_csv}...")
        with open(output_csv, mode="w", newline="") as file:
            writer = csv.DictWriter(file, fieldnames=[
                "Device", "Designator", "Power Loss (W)",
                "Top Temp (C)", "Bottom Temp (C)",
                "Temp Rise (C)", "Psi (K/W)"
            ])
            writer.writeheader()
            writer.writerows(results)

        print("Simulation and results processing complete.")
        return results

    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    finally:
        # Release AEDT
        ipk.release_desktop()
        print("IcePak simulation setup complete.")




In [ ]:
## Run Thermal Simulation

# Check if we have all required paths and data before running
if os.path.exists(project_folder) and os.path.exists(idf_file) and \
   os.path.exists(bdf_file) and os.path.exists(edb_file) and device_losses:

    # Run the simulation
    thermal_results = setup_solve_and_export_results(
        project_folder, idf_file, bdf_file, edb_file,
        device_losses, output_csv, aedt_version, ng_mode
    )

else:
    print("Please check your file paths and make sure the GaN losses data is loaded correctly.")
    thermal_results = None

Result Visualisation

In [ ]:
def visualize_thermal_results(results, gan_data):
    """
    Create visualizations of thermal simulation results.
    """
    if not results or not gan_data:
        print("No results to visualize.")
        return

    # Create a figure with multiple subplots
    fig = plt.figure(figsize=(15, 10))

    # 1. Temperature bar chart
    ax1 = fig.add_subplot(221)
    devices = [r["Device"] for r in results]
    top_temps = [r["Top Temp (C)"] for r in results]
    bottom_temps = [r["Bottom Temp (C)"] for r in results]

    x = np.arange(len(devices))
    width = 0.35

    ax1.bar(x - width/2, top_temps, width, label='Top Temperature')
    ax1.bar(x + width/2, bottom_temps, width, label='Bottom Temperature')

    ax1.set_ylabel('Temperature (°C)')
    ax1.set_title('Device Temperatures')
    ax1.set_xticks(x)
    ax1.set_xticklabels(devices)
    ax1.legend()

    # 2. Temperature rise bar chart
    ax2 = fig.add_subplot(222)
    temp_rises = [r["Temp Rise (C)"] for r in results]

    ax2.bar(devices, temp_rises, color='orange')
    ax2.set_ylabel('Temperature Rise (°C)')
    ax2.set_title('Device Temperature Rise')

    # 3. Thermal resistance (Psi) bar chart
    ax3 = fig.add_subplot(223)
    psi_values = [r["Psi (K/W)"] for r in results]

    ax3.bar(devices, psi_values, color='green')
    ax3.set_ylabel('Thermal Resistance (K/W)')
    ax3.set_title('Device Thermal Resistance (Psi)')

    # 4. Loss breakdown pie chart from GaN data
    ax4 = fig.add_subplot(224)

    labels = ['Turn-On Loss', 'Turn-Off Loss', 'Conduction Loss']
    sizes = [
        gan_data['losses']['turn_on_loss_W'],
        gan_data['losses']['turn_off_loss_W'],
        gan_data['losses']['conduction_loss_W']
    ]
    colors = ['#ff9999','#66b3ff','#99ff99']

    ax4.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
    ax4.axis('equal')
    ax4.set_title('GaN Power Loss Distribution')

    # Add some operating condition info as text
    plt.figtext(0.5, 0.01,
                f"Operating Conditions: Vin={gan_data['operating_conditions']['Vin']}V, "
                f"Vout={gan_data['operating_conditions']['Vout']}V, "
                f"Fsw={gan_data['operating_conditions']['Switching_frequency']/1000}kHz, "
                f"Efficiency={gan_data['efficiency']['efficiency_percent']}%",
                ha="center", fontsize=10, bbox={"facecolor":"orange", "alpha":0.1, "pad":5})

    plt.tight_layout(rect=[0, 0.05, 1, 0.95])
    plt.savefig('thermal_results_visualization.png')
    plt.show()

# Visualize results if we have them
if thermal_results and gan_data:
    visualize_thermal_results(thermal_results, gan_data)
    print(f"Visualization saved to 'thermal_results_visualization.png'")